In [1]:
import pandas as pd
from tqdm import tqdm
import json
import pickle
import time

In [2]:
construct_names = ['sexism', 'hatespeech']
construct = 'hatespeech'
pos_label = 'hate'
neg_label = 'not hate'

construct = 'sexism'
pos_label = 'sexist'
neg_label = 'non-sexist'

In [3]:
# ! pip install --upgrade openai

In [4]:
import openai
openai.api_key = "" # enter your openAI key

# list models
models = openai.Model.list()

In [5]:
# completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world!"}],n=3)

## Prompt Section

In [6]:
def create_prompt_label_sexism(original_sentence):
    
    prompt = 'Your task is to annotate if a piece of text is sexist or not. Read on for detailed explanations. \
            Something can be sexist: \
            because of its content (what the speaker believes) \
            because of its phrasing (because of the speaker’s choice of words) \
            Sexism due to content. A message may be sexist because of its content--in other words, because of what the speaker believes. For example, the speaker may express sexist attitudes towards gender inequality, either endorsing it (e.g. “some jobs are best left to men”), or antagonizing it (e.g. “the pay gap between genders does not exist, feminists should stop complaining”). Also, the speaker may express stereotypes (how genders are traditionally seen and compared to each other) and behavioral expectations (how individuals of a gender should behave according to traditional views). Sexism may also include positive stereotypes (e.g. “women are the best home cooks”), or target  men (e.g., “men should not cry”)! \
            Sexism due to phrasing. A message may be sexist simply because of how the speaker phrases it--independently from what general beliefs or attitudes the speaker holds. A message is sexist, for example, when it contains attacks, foul language, or derogatory depictions directed towards individuals because of their gender, e.g. by means of name-calling ("you bitch"), attacks ("I\'m going to kick her back to the kitchen"), objectification ("She\'s stupid but I\'d still do her"), inflammatory messages ("burn all women"). However, just because a message is aggressive or uses offensive language does not mean that it is sexist! \
            Sexist Examples: \
            “So I wore a dress and wig while bartending and when people asked what my costume was I said a bad driver #dassexist.” This post is sexist because it espouses gendered stereotypes, that women are allegedly not good at driving \
            “You’re a stupid bitch and I hope you die.” This post is sexist because of its phrasing, it contains a gendered slur (‘bitch’). \
            “Women’s sacrifices continue to ensure the future of this country’s sons.” This post espouses benevolent sexism. While such a sentiment might appear benign, benevolent sexism glorifies women and sees their worth only as extensions of their male relatives. \
            Non-sexist Examples \
            “F*ck you Jane ur so stupid” the content is offensive and Jane, the target, is likely a woman; however the message is not clearly sexist because it is unclear if the speaker’s animosity is because of Jane’s gender \
            With these instructions in mind and a piece of text, please reply with either of the three options and nothing else: 1) sexist, 2) non-sexist, 3) can\'t say. Please use the last option as little as possible and if you do, provide a one sentence explanation. Do not give an explanation if you pick \'sexist\' or \'non-sexist\'. We are ready to begin now. Is this sentence sexist based on the past guidelines: ' + original_sentence 
    return prompt

In [7]:
def create_prompt_label_hatespeech(original_sentence):
    
    prompt = 'In this task you have to determine if a piece of text contains hate speech or not. \
            The concept of hate speech refers to language or expression that aims to insult, offend, or threaten individuals or groups based on their identity. Different platforms, institutions, and scholars have offered slightly different definitions of hate speech. However, they share some commonalities, including that hate speech involves expressing something negative, such as contempt, disparagement, derogation, demonization, harm, or bias. The target of hate speech is the identity of a group, which can be based on ethnic origin, religion, gender, sexual orientation, race, ethnicity, nationality, or other characteristics.\
            Read the text carefully and decide if it contains hate speech.\
            If the text degrades, threatens, dehumanizes, or insults someone based on their race, religion, gender, sexual orientation, or any other protected characteristic, then mark it as hate speech.\
            If the text does not contain any hate speech, then mark it as not hateful.\
            Consider the context, speaker\'s intent, and audience, as hate speech can be explicit or implicit and can include sarcasm, humor, or irony.\
            If you are unsure whether a text contains hate speech, err on the side of caution and label it as hateful.\
            Aim for accuracy and consistency in your labeling.\
            Examples of content containing hate speech:\
            "All Muslims are terrorists." This post contains hate speech because it unfairly degrades an entire group of people based on their religion.\
            “I\m a huge fan of those that overtly express what we all think: jews are a plague and not welcome in our country.” This post contains hate speech because it promotes anti-Semitic beliefs while dehumanizing Jewish people.\
            "Transgender people are sick and perverted." This post contains hate speech because it stigmatizes a marginalized group of people based on their gender identity.\
            Examples of content not containing hate speech:\
            “I called him out for calling the new student a ‘tranny’.” This is counterspeech, i.e., it opposes the use of transphobic slurs rather than uses them to denigrate others.\
            With these instructions in mind and a piece of text, please reply with either of the three options and nothing else: 1) hate, 2) not hate, 3) can\'t say. Please use the last option as little as possible and if you do, provide a one sentence explanation. We are ready to begin now. Does this sentence contain hate speech based on the past guidelines: ' + original_sentence 
    return prompt

In [8]:
## test
test_text = "women are capable of commentating on football matches" #"women are weak and belong in the kitchen"
# result = openai.ChatCompletion.create(model="gpt-3.5-turbo",
#                                       messages=[{"role": "user", "content": create_prompt_label_hatespeech(test_text)}],
#                                       n=3)

## load test data

In [9]:
test_sets = ['in_domain', 'out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3',# 'out_of_domain_4', 
             'hatecheck'
]

In [10]:
test_set_data = {}

for construct in construct_names:
    test_set_data[construct] = {}
    for test_set in test_sets:
        test_path = '../../../data/data/%s/test/%s.csv' %(construct, test_set)
        test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t').head(10)

In [11]:
# ## do once and save

# for test_set in test_set_data['sexism']:
#     results = []
#     for _, row in test_set_data['sexism'][test_set].iterrows():
#         result = []
#         try:
#             responses = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_label_sexism(row['text'])}],n=3)
#         except:
#             time.sleep(60)
#             responses = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_label_sexism(row['text'])}],n=3)
#         for response in responses['choices']:
#             result.append(response['message']['content'])
#         results.append(result)
#     test_set_data['sexism'][test_set]['chatgpt_labels'] = results

In [12]:
# with open('../../designed_data/sexism_chatgpt_labels.pickle', 'wb') as handle:
#     pickle.dump(test_set_data['sexism'], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../designed_data/sexism_chatgpt_labels_all.pickle', 'rb') as handle:
    test_set_data['sexism'] = pickle.load(handle)

In [13]:
test_set_data['sexism'].keys()

dict_keys(['in_domain', 'out_of_domain', 'hatecheck', 'out_of_domain_2', 'out_of_domain_3'])

In [14]:
# remaining_test_sets = [#'in_domain', 'out_of_domain', 
#              'out_of_domain_2', 'out_of_domain_3',# 'out_of_domain_4', 
#              #'hatecheck'
# ]

# for construct in construct_names:
#     for test_set in remaining_test_sets:
#         test_path = '../../../data/data/%s/test/%s.csv' %(construct, test_set)
#         test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t')#.head(10)
        
# test_set_data['sexism'].keys()

In [15]:
# ## do once for the remaining and save

# for test_set in remaining_test_sets:
#     results = []
#     for _, row in tqdm(test_set_data['sexism'][test_set].iterrows(), total=test_set_data['sexism'][test_set].shape[0]):
#         result = []
#         try:
#             responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
#                                                      messages=[{"role": "user",
#                                                                 "content": create_prompt_label_sexism(row['text'])}],n=3)
#         except:
#             time.sleep(60)
#             try:
#                 responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
#                                                      messages=[{"role": "user",
#                                                                 "content": create_prompt_label_sexism(row['text'])}],n=3)
#             except:
#                 time.sleep(15*60)
#         for response in responses['choices']:
#             result.append(response['message']['content'])
#         results.append(result)
#     test_set_data['sexism'][test_set]['chatgpt_labels'] = results
    
# with open('../../designed_data/sexism_chatgpt_labels_all.pickle', 'wb') as handle:
#     pickle.dump(test_set_data['sexism'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
test_set_data['sexism']['out_of_domain_3']

,_id,text,sexism,chatgpt_labels
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",non-sexist,[Sexist. This message contains violent and der...
1,sexism2022_english-16993,"Then, she's a keeper. 😉",non-sexist,"[Sexist., Sexist., Sexist.]"
2,sexism2022_english-13149,This is like the Metallica video where the poo...,non-sexist,"[Non-sexist., Non-sexist., Non-sexist.]"
3,sexism2022_english-13021,woman?,non-sexist,[Can't say. There is no context or surrounding...
4,sexism2022_english-966,I bet she wished she had a gun,non-sexist,[Can't say. The sentence is ambiguous and lack...
...,...,...,...,...
19995,sexism2022_english-5228,girls really get fucked almost every weekend ?,non-sexist,"[sexist, Sexist., Sexist.]"
19996,sexism2022_english-10140,The hatred for moslems are Reasonable and Just...,non-sexist,"[Sexist., Sexist., Sexist.]"
19997,sexism2022_english-9726,Now this is a woman who gets it. 👆,non-sexist,"[Sexist., Sexist., Sexist.]"
19998,sexism2022_english-13365,“American Idol” finalist [USER] said nothing i...,non-sexist,"[Non-sexist., Non-sexist., Non-sexist.]"


In [17]:
test_set_data_unavail = {'sexism' : {}, 'hatespeech' : {}}
for test_set in test_set_data['sexism']:
    test_set_data_unavail['sexism'][test_set] = test_set_data['sexism'][test_set].copy()

In [18]:
import string 

count_no_label = {}

for test_set in test_set_data['sexism'].keys():
    count_no_label[test_set] = 0
    results = []
    results_un = []
    
    for _, row in test_set_data['sexism'][test_set].iterrows():
        label = row['chatgpt_labels'][0].lower().translate(str.maketrans('', '', string.punctuation))
        label = label.split(' ')[0]
        label_un = row['chatgpt_labels'][0].lower().translate(str.maketrans('', '', string.punctuation))
        label_un = label_un.split(' ')[0]
    
        if label == 'nonsexist':
            label = 'non-sexist'
            label_un = 'non-sexist'
        elif label == 'sexis' or label == 'benevolent':
            label = 'sexist'
            label_un = 'sexist'
        
        if label != 'sexist' and label != 'non-sexist':
            label_un = 'unavailable'
            label = 'non-sexist'
            count_no_label[test_set] += 1
    
        results.append(label)
        results_un.append(label_un)
    
    test_set_data['sexism'][test_set]['chatgpt_label_aggregate'] = results
    test_set_data_unavail['sexism'][test_set]['chatgpt_label_aggregate'] = results_un
    #[test_set]

In [19]:
[(i, count_no_label[i]/len(test_set_data['sexism'][i])) for i in count_no_label]

[('in_domain', 0.028594771241830064),
 ('out_of_domain', 0.03143189755529686),
 ('hatecheck', 0.003929273084479371),
 ('out_of_domain_2', 0.11258581235697941),
 ('out_of_domain_3', 0.055)]

In [20]:
'unavailable' in results_un

True

In [21]:
'unavailable' in test_set_data['sexism'][test_set]['chatgpt_label_aggregate'].values

False

In [22]:
for test_set in test_set_data_unavail['sexism'].keys():
    test_set_data_unavail['sexism'][test_set] = test_set_data_unavail['sexism'][test_set][test_set_data_unavail['sexism'][test_set]['chatgpt_label_aggregate'] \
                                                                                          != 'unavailable'] 

In [23]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, roc_curve, f1_score, accuracy_score
for test_set in test_set_data['sexism']:
    print(test_set)
    print('unavailable' in test_set_data['sexism'][test_set]['chatgpt_label_aggregate'].values)
    print('unavailable' in test_set_data['sexism'][test_set]['sexism'].values)
    print(classification_report(test_set_data['sexism'][test_set]['sexism'],
                                test_set_data['sexism'][test_set]['chatgpt_label_aggregate']))

in_domain
False
False
              precision    recall  f1-score   support

  non-sexist       0.92      0.60      0.72       690
      sexist       0.64      0.93      0.76       534

    accuracy                           0.74      1224
   macro avg       0.78      0.77      0.74      1224
weighted avg       0.80      0.74      0.74      1224

out_of_domain
False
False
              precision    recall  f1-score   support

  non-sexist       0.79      0.61      0.69      1800
      sexist       0.66      0.82      0.73      1636

    accuracy                           0.71      3436
   macro avg       0.73      0.72      0.71      3436
weighted avg       0.73      0.71      0.71      3436

hatecheck
False
False
              precision    recall  f1-score   support

  non-sexist       1.00      0.52      0.69       136
      sexist       0.85      1.00      0.92       373

    accuracy                           0.87       509
   macro avg       0.93      0.76      0.80       509
weig

In [24]:
for test_set in test_set_data['sexism']:
    print(test_set)
    print(classification_report(test_set_data_unavail['sexism'][test_set]['sexism'],
                                test_set_data_unavail['sexism'][test_set]['chatgpt_label_aggregate']))

in_domain
              precision    recall  f1-score   support

  non-sexist       0.94      0.58      0.72       664
      sexist       0.64      0.95      0.77       525

    accuracy                           0.74      1189
   macro avg       0.79      0.77      0.74      1189
weighted avg       0.81      0.74      0.74      1189

out_of_domain
              precision    recall  f1-score   support

  non-sexist       0.80      0.60      0.68      1718
      sexist       0.66      0.84      0.74      1610

    accuracy                           0.71      3328
   macro avg       0.73      0.72      0.71      3328
weighted avg       0.73      0.71      0.71      3328

hatecheck
              precision    recall  f1-score   support

  non-sexist       1.00      0.51      0.68       134
      sexist       0.85      1.00      0.92       373

    accuracy                           0.87       507
   macro avg       0.93      0.76      0.80       507
weighted avg       0.89      0.87      0

In [25]:
test_set_data['sexism']['in_domain']

,_id,text,sexism,chatgpt_labels,chatgpt_label_aggregate
0,5d654919ce2c6bd733477d59,Maybe the girls should have less tickets on th...,sexist,"[Sexist., Sexist., Sexist.]",sexist
1,5d654904ce2c6bd733475d0c,ABCDEFG means A Boy Can Do Everything For a ...,non-sexist,[Sexist. The sentence reinforces gender stereo...,sexist
2,5d654914ce2c6bd7334772b3,"Today in ""things GamerGate does to try to stop...",non-sexist,"[Non-sexist., Non-sexist., non-sexist]",non-sexist
3,5d654901ce2c6bd733475c0e,Nothing like a man willing to spoil you. 😌 how...,non-sexist,"[Sexist., Sexist., Sexist.]",sexist
4,5ddc4fe8174b03a4c6aaf74b,Having a job means having a life of your own,non-sexist,"[non-sexist, Non-sexist., non-sexist]",non-sexist
...,...,...,...,...,...
1219,5d654922ce2c6bd733479046,I find it strange there are females working on...,sexist,"[Sexist., Sexist., Sexist.]",sexist
1220,5d654924ce2c6bd73347955f,this election I'm just voting for the candidat...,non-sexist,"[Sexist., Sexist., Sexist.]",sexist
1221,5d65490ece2c6bd733476bd8,Dont be disheartened emilie n sherri! #MKR,non-sexist,"[Non-sexist., Non-sexist., Non-sexist.]",non-sexist
1222,5d65491ece2c6bd7334784aa,RT @eoind87 Great comment from a lad in class ...,sexist,"[Sexist., Sexist., Sexist.]",sexist


In [26]:
test_sets = ['in_domain', 'out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3', 'out_of_domain_4', 
             'hatecheck']

In [27]:
# test_set_data = {}

# for construct in ['hatespeech']:
#     test_set_data[construct] = {}
#     for test_set in test_sets:
#         test_path = '../../../../socialCAD_/data/data/%s/test/%s.csv' %(construct, test_set)
#         test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t')

In [28]:
# ## do once and save

# for test_set in test_set_data['hatespeech']:
#     results = []
#     for _, row in test_set_data['hatespeech'][test_set].iterrows():
#         result = []
#         try:
#             responses = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_label_hatespeech(row['text'])}],n=3)
#         except:
#             time.sleep(60)
#             responses = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_label_hatespeech(row['text'])}],n=3)
#         for response in responses['choices']:
#             result.append(response['message']['content'])
#         results.append(result)
#     test_set_data['hatespeech'][test_set]['chatgpt_labels'] = results

In [29]:
# with open('../../designed_data/hatespeech_chatgpt_labels.pickle', 'wb') as handle:
#     pickle.dump(test_set_data['hatespeech'], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../designed_data/hatespeech_chatgpt_labels.pickle', 'rb') as handle:
    test_set_data['hatespeech'] = pickle.load(handle)    

In [30]:
test_set_data['hatespeech'][test_set].groupby('hatespeech').size()

hatespeech
hate         5256
not hate    17067
dtype: int64

In [31]:
for test_set in test_set_data['hatespeech']:
    test_set_data_unavail['hatespeech'][test_set] = test_set_data['hatespeech'][test_set].copy()

In [32]:
count_no_label = {}
for test_set in  test_set_data['hatespeech']:
    count_no_label[test_set] = 0
    results = []
    results_un = []
    for _, row in test_set_data['hatespeech'][test_set].iterrows():
        label = row['chatgpt_labels'][0].lower().translate(str.maketrans('', '', string.punctuation))
        label = label.split(' ')[0]
        label_un = label
        if label == 'not':
            label = 'not hate'
            label_un = 'not hate'
        
        if label != 'hate' and label != 'not hate':
            label = 'not hate'
            label_un = 'unavailable'
            count_no_label[test_set] += 1

        results.append(label)
        results_un.append(label_un)
    test_set_data['hatespeech'][test_set]['chatgpt_label_aggregate'] = results
    test_set_data_unavail['hatespeech'][test_set]['chatgpt_label_aggregate'] = results_un
    #[test_set]

In [33]:
[(i, count_no_label[i]/len(test_set_data['hatespeech'][i])) for i in count_no_label]

[('in_domain', 0.12620320855614972),
 ('out_of_domain', 0.5188003315963998),
 ('out_of_domain_2', 0.076),
 ('out_of_domain_3', 0.34220310890113337),
 ('out_of_domain_4', 0.06581013561741614),
 ('hatecheck', 0.05472103004291846)]

In [34]:
for test_set in test_set_data['hatespeech'].keys():
    test_set_data_unavail['hatespeech'][test_set] = test_set_data_unavail['hatespeech'][test_set][test_set_data_unavail['hatespeech'][test_set]['chatgpt_label_aggregate'] \
                                                                                                  != 'unavailable'] 

In [35]:
for test_set in test_set_data_unavail['hatespeech']:
    print(test_set)
    print(classification_report(test_set_data_unavail['hatespeech'][test_set]['hatespeech'],
                                test_set_data_unavail['hatespeech'][test_set]['chatgpt_label_aggregate']))

in_domain
              precision    recall  f1-score   support

        hate       0.70      0.83      0.76       427
    not hate       0.77      0.62      0.68       390

    accuracy                           0.73       817
   macro avg       0.73      0.72      0.72       817
weighted avg       0.73      0.73      0.72       817

out_of_domain
              precision    recall  f1-score   support

        hate       0.64      0.89      0.75      5069
    not hate       0.94      0.78      0.85     11184

    accuracy                           0.81     16253
   macro avg       0.79      0.83      0.80     16253
weighted avg       0.85      0.81      0.82     16253

out_of_domain_2
              precision    recall  f1-score   support

        hate       0.61      0.86      0.71      1187
    not hate       0.85      0.58      0.69      1585

    accuracy                           0.70      2772
   macro avg       0.73      0.72      0.70      2772
weighted avg       0.75      0.70 

In [36]:
for test_set in test_set_data['hatespeech']:
    print(test_set)
    print(classification_report(test_set_data['hatespeech'][test_set]['hatespeech'],
                                test_set_data['hatespeech'][test_set]['chatgpt_label_aggregate']))

in_domain
              precision    recall  f1-score   support

        hate       0.70      0.75      0.73       471
    not hate       0.73      0.68      0.70       464

    accuracy                           0.71       935
   macro avg       0.72      0.71      0.71       935
weighted avg       0.72      0.71      0.71       935

out_of_domain
              precision    recall  f1-score   support

        hate       0.64      0.31      0.42     14614
    not hate       0.62      0.87      0.73     19162

    accuracy                           0.63     33776
   macro avg       0.63      0.59      0.57     33776
weighted avg       0.63      0.63      0.59     33776

out_of_domain_2
              precision    recall  f1-score   support

        hate       0.61      0.81      0.70      1260
    not hate       0.82      0.62      0.71      1740

    accuracy                           0.70      3000
   macro avg       0.71      0.72      0.70      3000
weighted avg       0.73      0.70 

In [37]:
# with open('../../designed_data/hatespeech_chatgpt_labels.pickle', 'wb') as handle:
#     pickle.dump(test_set_data['hatespeech'], handle, protocol=pickle.HIGHEST_PROTOCOL)